In [ ]:
# scvelo virtual environment

import scvelo as scv
import numpy as np
import scanpy as sc
import math

In [ ]:
import subprocess

# List all installed packages with their versions
def list_installed_packages():
    result = subprocess.run(['pip', 'list'], stdout=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))

# List all installed packages with their versions in requirements format
def freeze_installed_packages():
    result = subprocess.run(['pip', 'freeze'], stdout=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))

# Execute the functions
list_installed_packages()
freeze_installed_packages()


In [ ]:
def generate_negative_binomial_matrix(mean_matrix, overdispersion):
    # Ensure overdispersion is positive
    if overdispersion <= 0:
        raise ValueError("Overdispersion parameter must be positive.")
    
    # Calculate the parameters r and p for each element in the matrix
    r_matrix = np.full(mean_matrix.shape, overdispersion)
    p_matrix = r_matrix / (r_matrix + mean_matrix)
    
    # Generate the negative binomial random variables for each element in the matrix
    negative_binomial_matrix = np.random.negative_binomial(r_matrix, p_matrix)
    
    return negative_binomial_matrix

In [ ]:
def generate_data(alpha,
                  beta,
                  gamma,
                  max_thre,
                  n_obs,
                  n_vars,
                  noise_level,
                  overdispersion,
                  seed,
                  t_max):
    # generate S with Gaussian noise
    adata = scv.datasets.simulation(n_obs=n_obs,
                                    n_vars=n_vars,
                                    t_max=t_max,
                                    alpha=alpha,
                                    beta=beta,
                                    gamma=gamma,
                                    noise_level=noise_level,
                                    random_seed=seed)
    S = adata.layers['spliced']
    U = adata.layers['unspliced']
    adata.layers['true_velocity'] = beta*U-gamma*S
    S_lam = math.e**(math.log(max_thre)*S/np.max(S)) - 1
    U_lam = math.e**(math.log(max_thre)*U/np.max(U)) - 1

    np.random.seed(seed)
    S_poi = generate_negative_binomial_matrix(mean_matrix=S_lam,
                                             overdispersion=overdispersion)
    U_poi = generate_negative_binomial_matrix(mean_matrix=U_lam,
                                             overdispersion=overdispersion)
    adata.layers['spliced'] = S_poi.copy()
    adata.layers['unspliced'] = U_poi.copy()
    adata.X = S_poi.copy() # need to make sure X is the same as spliced matrix

    # List of quantiles you want to find
    quantiles = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
    # Find the quantiles of the flattened array
    quantile_results = np.quantile(adata.X, quantiles)
    print(f"The quantiles {quantiles} of the matrix are: {quantile_results}")

    # https://github.com/theislab/scvelo/issues/1212 
    
    scv.pp.log1p(adata)
    sc.pp.pca(adata)
    sc.pp.neighbors(adata, n_pcs=30, n_neighbors=30)
    scv.pp.moments(adata, n_pcs=None, n_neighbors=None)

    return adata


In [ ]:
n_obs = 1000
n_vars = 100
t_max = 25
alpha = 5
beta = .3
gamma = .5
noise_level = 2
seed = 520
max_thre = 100
overdispersion = 1000 # the larger means the more like a Poisson

# True velocity

In [ ]:
data = generate_data(alpha=alpha,
                     beta=beta,
                     gamma=gamma,
                     max_thre=max_thre,
                     n_obs=n_obs,
                     n_vars=n_vars,
                     noise_level=noise_level,
                     overdispersion=overdispersion,
                     seed=seed,
                     t_max=t_max)

In [ ]:
scv.tl.recover_dynamics(data,t_max=25)
scv.tl.velocity_graph(data,vkey="true_velocity")

In [ ]:
scv.pl.velocity_embedding_grid(data,
                               basis='pca',
                               color="true_t",
                               vkey="true_velocity",
                               arrow_length=2,
                               arrow_size=2,
                               min_mass=10)

In [ ]:
scv.pl.velocity_embedding_stream(data,
                               basis='pca',
                               color="true_t",
                               vkey="true_velocity")

# Now for scVelo

In [ ]:
data2 = generate_data(alpha=alpha,
                      beta=beta,
                      gamma=gamma,
                      max_thre=max_thre,
                      n_obs=n_obs,
                      n_vars=n_vars,
                      noise_level=noise_level,
                      overdispersion=overdispersion,
                      seed=seed,
                      t_max=t_max)

In [ ]:
scv.tl.recover_dynamics(data2)

In [ ]:
scv.tl.velocity(data2, mode='dynamical')

In [ ]:
scv.tl.velocity_graph(data2)

In [ ]:
scv.pl.velocity_embedding_grid(data2,
                               basis='pca',
                               color="true_t",
                               arrow_length=2,
                               arrow_size=2,
                               min_mass=10)

In [ ]:
scv.pl.velocity_embedding_stream(data2,
                               basis='pca',
                               color="true_t")

In [ ]:
scv.tl.velocity_confidence(data2)
keys = 'velocity_confidence'
scv.pl.scatter(data2, c=keys, cmap='coolwarm', perc=[5, 95])